## Leetcode Practice Problems

### Design Log Aggregation System
* overview
  + Design the LogAggregator class:
    + LogAggregator(int machines, int services) Initializes the object with machines and services representing the number of machines and services in the datacenter, respectively.
    + void pushLog(int logId, int machineId, int serviceId, String message) Adds a log with id logId notifying that the machine machineId sent a string message while executing the service serviceId.
    + List\<Integer\> getLogsFromMachine(int machineId) Returns a list of ids of all logs added by machine machineId.
    + List\<Integer\> getLogsOfService(int serviceId) Returns a list of ids of all logs added while running service serviceId on any machine.
    + List\<String\> search(int serviceId, String searchString) Returns a list of messages of all logs added while running service serviceId where the message of the log contains searchString as a substring.  
  + Note that:
    + The entries in each list should be in the order they were added, i.e., the older logs should precede the newer logs.
    + A machine can run multiple services more than once. Similarly, a service can be run on multiple machines.
    + All logId may not be ordered.
    + A substring is a contiguous sequence of characters within a string.   
* algorithm
  + First, we need to have an ordered data structure for both machine list and service list
    + getLogsFromMachine and getLogsOfService both return an ordered list of all the log ids
  + search function returns a list of message in logs that contains the searchString as a substring
  + based on the above two requirements, we can set two lists, one for each machine id, the second for each service id to store the log ids added to each of the list when pushLog function is executed
  + to store the log messages, we use a dictionary from log id to log message. When search function is executed, we first fetch a list of log id, and then check for each log id, if its message contains the searchString
  
* solution
  + in the given solution from Leetcode, instead of using lists to store the mapping from service and machine ids to log ids, the solutoin used defaultdict, which is basically the same as the lists we used here since the element index is used as the key to the dictionary. Notice that we have a fixed number of machines and services, and therefore, each service and machine id are guaranteed to be within the index ranges of the lists
  + In addition, in search function, accssing to each service item by list index is also O(1), the same as default dict.

In [2]:
# our solution using lists and dictionary
from typing import List
class LogAggregator:

    def __init__(self, machines: int, services: int):
        self.machine_list = [[] for _ in range(machines)]
        self.service_list = [[] for _ in range(services)]
        self.logs = {}
        

    def pushLog(self, logId: int, machineId: int, serviceId: int, message: str) -> None:
        self.machine_list[machineId].append(logId)
        self.service_list[serviceId].append(logId)
        self.logs[logId] = message        

    def getLogsFromMachine(self, machineId: int) -> List[int]:
        return self.machine_list[machineId]        

    def getLogsOfService(self, serviceId: int) -> List[int]:
        return self.service_list[serviceId]
        

    def search(self, serviceId: int, searchString: str) -> List[str]:
        rs = []
        for log_id in self.service_list[serviceId]:
            if searchString in self.logs[log_id]:
                rs.append(self.logs[log_id])
            
        return rs    
            
        


# Your LogAggregator object will be instantiated and called as such:
# obj = LogAggregator(machines, services)
# obj.pushLog(logId,machineId,serviceId,message)
# param_2 = obj.getLogsFromMachine(machineId)
# param_3 = obj.getLogsOfService(serviceId)
# param_4 = obj.search(serviceId,searchString)

In [3]:
# official solution give by leetcode using defaultdict
class LogAggregator:

    def __init__(self, machines, services):
        self.machines = machines
        self.services = services
        self.logs = defaultdict(str)
        self.logsForMachine = defaultdict(list)
        self.logsForService = defaultdict(list)

    def pushLog(self, logId: int ,machineId: int, serviceId: int, message: str) -> None:
        self.logs[logId] = message
        self.logsForMachine[machineId].append(logId)
        self.logsForService[serviceId].append(logId)

    def getLogsFromMachine(self, machineId: int) -> List[int]:
        return self.logsForMachine[machineId]

    def getLogsOfService(self, serviceId: int) -> List[int]:
        return self.logsForService[serviceId]

    def search(self, service: int, searchString: str) -> List[str]:
        filteredLogs = []
        for logId in self.logsForService[service]:
            if searchString in self.logs[logId]:
                filteredLogs.append(self.logs[logId])
        return filteredLogs

### Design a Rate Limiting System
* Overview
  + A Rate Limiting System can allow a maximum of n requests every t seconds, using an implementation similar to the sliding window algorithm.
  + Given two positive integers n and t, and a non-decreasing stream of integers representing the timestamps of requests, implement a data structure that can check if each request is allowed or not.
  + Implement the RateLimiter class:
    + RateLimiter(int n, int t) Initializes the RateLimiter object with an empty stream and two integers n and t.
    + boolean shouldAllow(int timestamp) Returns true if the current request with timestamp timestamp is allowed, otherwise returns false. 
  + Note that while checking if the current request should be allowed, only the timestamps of requests previously allowed are considered.
* algorithm
  + we need a data structure that can accomodate at most n slots within a fixed time window of t
  + given a time stamp, we need to check if within the time window t, do we have enough time slots to accomodate the current request
  + we use a deque data structure, each time when a request comes, we first popleft all the records beyond the current time stamp - t, and check if we have enough time slot (the nubmer of the remaining items should be less than n), if so, we insert the time slot to the queue and return True, otherwise, we just returns False

In [ ]:
# our solution using deque
class RateLimiter:

    def __init__(self, n: int, t: int):
        self.queue = deque()
        self.capacity = n
        self.interval = t        

    def shouldAllow(self, timestamp: int) -> bool:
        # we only care about the requests having timestamp >= timestamp -self.interval+1
        while self.queue and self.queue[0] < timestamp - self.interval + 1:
            self.queue.popleft()
            
        # if we still have space with all requests having timestamps >= timestamp-self.interval +1
        # we add this timestamp to the queue and return True. Otherwise, return Fasle and ignore the request
        if not self.queue or len(self.queue) < self.capacity:
            self.queue.append(timestamp)
            return True
        
        return False
            
# Your RateLimiter object will be instantiated and called as such:
# obj = RateLimiter(n, t)
# param_1 = obj.shouldAllow(timestamp)     

In [ ]:
# official solution from leetcode
class RateLimiter:

    def __init__(self, N: int, T: int):
        self.N = N
        self.T = T
        self.allowedRequests = deque()

    # Returns true if there were less than N requests that we allowed in the last T seconds.
    def shouldAllow(self, epochTimeInSeconds: int) -> bool:
        if not self.allowedRequests:
            self.allowedRequests.appendleft(epochTimeInSeconds)
            return True
        while (self.allowedRequests and epochTimeInSeconds - self.allowedRequests[-1] >= self.T):
            self.allowedRequests.pop()
        if len(self.allowedRequests) >= self.N:
            return False
        self.allowedRequests.appendleft(epochTimeInSeconds)
        return True

### Design a Load Distributor
* Overview
  + Design a simple load distributor for a data center that can do the following:
    + Add and remove machines from the cluster.
    + Add applications to run on a machine.
    + Stop applications that are running on a machine.
    + Return a list of the applications running on a machine.
  + Implement the DCLoadBalancer class:
    + DCLoadBalancer() Initializes the object.
    + void addMachine(int machineId, int capacity) Registers a machine with the given machineId and maximum capacity.
    + void removeMachine(int machineId) Removes the machine with the given machineId. All applications running on this machine are automatically reallocated to other machines in the same order as they were added to this machine. The applications should be reallocated in the same manner as addApplication.
    + int addApplication(int appId, int loadUse) Allocates an application with the given appId and loadUse to the machine with the largest remaining capacity that can handle the additional request. If there is a tie, the machine with the lowest ID is used. Returns the machine ID that the application is allocated to. If no machine can handle the request, return -1.
    + void stopApplication(int appId) Stops and removes the application with the given appId from the machine it is running on, freeing up the machine's capacity by its corresponding loadUse. If the application does not exist, nothing happens.
    + List\<Integer\> getApplications(int machineId) Returns a list of application IDs running on a machine with the given machineId in the order in which they were added. If there are more than 10 applications, return only the first 10 IDs.
    
* Example

Input                        
["DCLoadBalancer", "addMachine", "addMachine", "addMachine", "addMachine", "addApplication", "addApplication", "addApplication", "addApplication", "getApplications", "addMachine", "addApplication", "stopApplication", "addApplication", "getApplications", "removeMachine", "getApplications"]
[[], [1, 1], [2, 10], [3, 10], [4, 15], [1, 3], [2, 11], [3, 6], [4, 5], [2], [5, 10], [5, 5], [3], [6, 5], [4], [4], [2]]
Output
[null, null, null, null, null, 4, 4, 2, 3, [3], null, 5, null, 2, [1, 2], null, [6, 1]]

Explanation                                         
DCLoadBalancer dCLoadBalancer = new DCLoadBalancer();                               
dCLoadBalancer.addMachine(1, 1); // Capacity Left: [1]                              
dCLoadBalancer.addMachine(2, 10); // Capacity Left: [1,10]                             
dCLoadBalancer.addMachine(3, 10); // Capacity Left: [1,10,10]                                
dCLoadBalancer.addMachine(4, 15); // Capacity Left: [1,10,10,15]                                 
dCLoadBalancer.addApplication(1, 3); // return 4, Capacity Left: [1,10,10,12]                  
                                     // Machine 4 had the largest capacity left at 15.               
dCLoadBalancer.addApplication(2, 11); // return 4, Capacity Left: [1,10,10,1]                  
                                      // Machine 4 had the largest capacity left at 12.             
dCLoadBalancer.addApplication(3, 6); // return 2, Capacity Left: [1,4,10,1]               
                                     // Machine 2 and 3 had the same largest capacity but machine 2 has the lower ID.
dCLoadBalancer.addApplication(4, 5); // return 3, Capacity Left: [1,4,5,1]                     
                                     // Machine 3 had the largest capacity at 10.                            
dCLoadBalancer.getApplications(2); // return [3], Machine 2 only has application 3.                
dCLoadBalancer.addMachine(5, 10); // Capacity Left: [1,4,5,1,10]                           
dCLoadBalancer.addApplication(5, 5); // return 5, Capacity Left: [1,4,5,1,5]
                                     // Machine 5 had the largest capacity at 10.            
dCLoadBalancer.stopApplication(3); // Capacity Left: [1,10,5,1,5],                          
                                   // Application 3 was running on machine 2.                   
dCLoadBalancer.addApplication(6, 5); // return 2, Capacity Left: [1,5,5,1,5]                   
                                     // Machine 2 had the largest capacity at 10.                   
dCLoadBalancer.getApplications(4); // return [1, 2], Machine 4 has applications 1 and 2.       
dCLoadBalancer.removeMachine(4); // Capacity Left: [1,2,5,-,5]           
                                 // Machine 4 had applications 1 and 2.         
                                 // Application 1 has a load of 3 and is added to machine 2.    
                                 // Application 2 has a load of 11 and cannot be added to any machine so it is removed.        
dCLoadBalancer.getApplications(2); // return [6, 1], Machine 2 has applications 6 and 1.      

* Algorithm
  + 

In [4]:
from heapq import heappop, heappush, heapify
from typing import List

class Application:
    def __init__(self, app_id: int, load: int, machine_id : int=-1):
        self.app_id = app_id
        self.load = load
        self.machine_id = machine_id
        
    def setMachineId(self, machine_id: int) -> None:
        self.machine_id = machine_id
        
    def getLoad(self) -> int:
        return self.load
    
    def getMachineId(self) -> int:
        return self.machine_id
    
    def getAppId(self) -> int:
        return self.app_id
    
class Machine:
    def __init__(self, m_id: int, capacity: int):
        self.id = m_id
        self.apps = []
        self.remain = capacity
        
    def __lt__(self, other) -> bool:
        if self.remain == other.remain:
            return self.id < other.id
        return other.remain < self.remain
    
    def addApp(self, app: 'Application') -> bool:
        if self.remain < app.getLoad():
            return False
        
        self.apps.append(app.getAppId())
        self.remain -= app.getLoad()
        app.setMachineId(self.id)
        return True
    
    def removeApp(self, app: 'Application' ) -> None:
        self.apps.remove(app.getAppId())
        self.remain += app.getLoad()
        app.setMachineId(-1) 
        
    def getApp(self) -> List['Applicatoin']:
        return self.apps
    
    def getRemain(self) -> int:
        return self.remain
    
    def getId(self) -> int:
        return self.id

class DCLoadBalancer:

    def __init__(self):
        # machine objs list to maintain machine with highest available capacity
        self.heap = []
        
        # dictionary to maintain app ids with application objs
        self.apps = {}
        
        # dictionary to maintain machine ids with machine objs
        self.machines = {}

    def addMachine(self, machineId: int, capacity: int) -> None:
        
        # construct machine obj
        machine = Machine(machineId, capacity)
        
        # push machine obj to heap
        heappush(self.heap, machine)
        
        # add machine obj to dictionary with machine id as the key
        # we can update machine from this dictionary with its key
        self.machines[machineId] = machine
                
    def removeMachine(self, machineId: int) -> None:
        # if the machine id doesn't exist in dictionary, it has been deleted
        if machineId not in self.machines:
            return
        
        # if the heap top contains the machine to be removed, pop the machine from heap
        if self.heap[0].id == machineId:
            heappop(self.heap)
           
        # get all app ids from the machine to be removed to tranfer 
        # delete the entry of this machine from machine dictionary
        apps = self.machines[machineId].getApp()
        del self.machines[machineId]    
        
        # add each app to the max capacity available machines
        for app in apps:
            self.addApplication(app, self.apps[app].getLoad())            

    def addApplication(self, appId: int, loadUse: int) -> int:
                
        # remove all the deleted machines from heap
        while self.heap and self.heap[0].id not in self.machines:
            heappop(self.heap)        
        
        if not self.heap:
            return -1      
        
        # if the app size is bigger than the max available capacity machine, return -1
        if self.heap[0].getRemain() < loadUse:
            return -1
        
        # otherwise, pop the max capacity available machine (successor) from heap, 
        # consturct app obj, add the app to the successor, and push it back to heap
        successor = heappop(self.heap)
        app = Application(appId, loadUse)
        successor.addApp(app)        
        heappush(self.heap, successor)
       
        # register the app to apps dictionary
        self.apps[appId] = app
            
        return successor.getId()   
        

    def stopApplication(self, appId: int) -> None:
        # if the app has been deleted, return
        if appId not in self.apps:
            return
        
        # get the app obj to be deleted from dictionary, and delete its entry
        app = self.apps[appId]
        del self.apps[appId]
        
        # remove the app obj from the corresponding machine obj
        # then heapify the self.heap to maintain the max heap
        self.machines[app.getMachineId()].removeApp(app)       
        heapify(self.heap)       
        

    def getApplications(self, machineId: int) -> List[int]:
        
        # return the first 10 app obj's id for a specific machineId
        return self.machines[machineId].getApp()[:10]
        


# Your DCLoadBalancer object will be instantiated and called as such:
# obj = DCLoadBalancer()
# obj.addMachine(machineId,capacity)
# obj.removeMachine(machineId)
# param_3 = obj.addApplication(appId,loadUse)
# obj.stopApplication(appId)
# param_5 = obj.getApplications(machineId)